In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client(project='code-for-planet')

In [ ]:
impute_all_zeros_query = """
CREATE OR REPLACE TABLE `code-for-planet.emission_db.emissions_imputation_candidates` AS

SELECT
    *,
    -- Flag different zero patterns for targeted treatment
    CASE
        WHEN area_burned = 0 AND ECO2 = 0 AND ECO = 0 AND ECH4 = 0 AND consumed_fuel = 0
        THEN 'all_zero_missing_data'

        WHEN area_burned = 0 AND (ECO2 > 0 OR ECO > 0 OR ECH4 > 0)
        THEN 'suspicious_emissions_without_area'

        WHEN area_burned > 0 AND ECO2 = 0 AND ECO = 0 AND ECH4 = 0
        THEN 'suspicious_area_without_emissions'

        WHEN ECO2 > 0 AND ECO > 0 AND ECH4 > 0 AND area_burned > 0
        THEN 'complete_data'

        ELSE 'partial_data'
    END AS imputation_priority,

    -- Count how many emission variables are zero
    (CASE WHEN area_burned = 0 THEN 1 ELSE 0 END +
     CASE WHEN ECO2 = 0 THEN 1 ELSE 0 END +
     CASE WHEN ECO = 0 THEN 1 ELSE 0 END +
     CASE WHEN ECH4 = 0 THEN 1 ELSE 0 END +
     CASE WHEN consumed_fuel = 0 THEN 1 ELSE 0 END) AS zero_variable_count

FROM `code-for-planet.emission_db.emissions_db_imputation`
"""

job = client.query(impute_all_zeros_query)
job.result()

## Check the priority distribution

In [ ]:
imputation_coverage_query = """
SELECT
    imputation_priority,
    COUNT(*) as record_count,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() as percentage
FROM `code-for-planet.emission_db.emissions_imputation_candidates`
GROUP BY imputation_priority
ORDER BY record_count DESC
"""

priority_distribution = client.query(imputation_coverage_query).to_dataframe()
priority_distribution

,imputation_priority,record_count,percentage
0,complete_data,4761421,65.633726
1,suspicious_emissions_without_area,1199151,16.529676
2,all_zero_missing_data,1113481,15.348760
3,suspicious_area_without_emissions,180481,2.487837


In [ ]:
hotdeck_imputation_query = """
CREATE OR REPLACE TABLE `code-for-planet.emission_db.emissions_hotdeck_imputed` AS

WITH donor_matches AS (
    SELECT
        missing.global_id AS missing_record_id,
        donor.ECO2 AS donor_ECO2,
        donor.ECO AS donor_ECO,
        donor.ECH4 AS donor_ECH4,
        donor.area_burned AS donor_area_burned,
        donor.consumed_fuel AS donor_consumed_fuel,

        -- Much simpler ranking (no spatial distance!)
        ROW_NUMBER() OVER (
            PARTITION BY missing.global_id
            ORDER BY
                -- Same fire event gets highest priority
                CASE WHEN missing.global_fire_event_id = donor.global_fire_event_id THEN 1 ELSE 2 END,
                -- Then exact fuel and cover match
                CASE WHEN missing.covertype = donor.covertype AND missing.fuelcode = donor.fuelcode THEN 1 ELSE 2 END,
                -- Then similar weather
                ABS(missing.fuel_moisture_class - donor.fuel_moisture_class),
                -- Use DOY (day of year) as rough temporal proximity
                ABS(missing.doy - donor.doy),
                -- Random tie-breaker
                RAND()
        ) AS donor_rank

    FROM `code-for-planet.emission_db.emissions_imputation_candidates` missing
    JOIN `code-for-planet.emission_db.emissions_imputation_candidates` donor
        ON missing.global_id != donor.global_id
        AND missing.year = donor.year  -- Same year only
        AND donor.imputation_priority = 'complete_data'  -- Donor has good data
        -- Add these filters to reduce join size
        AND missing.covertype = donor.covertype  -- Must match vegetation
        AND missing.fuelcode = donor.fuelcode     -- Must match fuel type

    WHERE missing.imputation_priority = 'all_zero_missing_data'  -- Target missing data
)

SELECT
    e.*,
    dm.donor_ECO2,
    dm.donor_ECO,
    dm.donor_ECH4,
    dm.donor_area_burned,
    dm.donor_consumed_fuel,

    -- Apply hot-deck imputation
    COALESCE(NULLIF(e.ECO2, 0), dm.donor_ECO2, 0) AS ECO2_hotdeck,
    COALESCE(NULLIF(e.ECO, 0), dm.donor_ECO, 0) AS ECO_hotdeck,
    COALESCE(NULLIF(e.ECH4, 0), dm.donor_ECH4, 0) AS ECH4_hotdeck,
    COALESCE(NULLIF(e.area_burned, 0), dm.donor_area_burned, 0) AS area_burned_hotdeck,
    COALESCE(NULLIF(e.consumed_fuel, 0), dm.donor_consumed_fuel, 0) AS consumed_fuel_hotdeck,

    CASE
        WHEN dm.donor_ECO2 IS NOT NULL THEN 'hotdeck_imputed'
        WHEN e.imputation_priority = 'complete_data' THEN 'original_data'
        ELSE 'needs_further_imputation'
    END AS imputation_status

FROM `code-for-planet.emission_db.emissions_imputation_candidates` e
LEFT JOIN donor_matches dm
    ON e.global_id = dm.missing_record_id
    AND dm.donor_rank = 1  -- Use best donor only
"""

job = client.query(hotdeck_imputation_query)
job.result()

In [ ]:
hotdeck_results_query = """
SELECT
    imputation_status,
    COUNT(*) as record_count,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() as percentage,
    AVG(CASE WHEN ECO2_hotdeck > 0 THEN ECO2_hotdeck END) as avg_ECO2_imputed,
    AVG(CASE WHEN area_burned_hotdeck > 0 THEN area_burned_hotdeck END) as avg_area_burned_imputed
FROM `code-for-planet.emission_db.emissions_hotdeck_imputed`
GROUP BY imputation_status
ORDER BY record_count DESC
"""

hotdeck_results = client.query(hotdeck_results_query).to_dataframe()
hotdeck_results

,imputation_status,record_count,percentage,avg_ECO2_imputed,avg_area_burned_imputed
0,original_data,4761421,65.633726,2912.161777,62500.0
1,needs_further_imputation,2493109,34.366218,3175.060880,62500.0
2,hotdeck_imputed,4,0.000055,1428.356422,62500.0


In [ ]:
imputation_success_query = """
SELECT
    imputation_priority,
    imputation_status,
    COUNT(*) as record_count,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY imputation_priority) as pct_within_priority
FROM `code-for-planet.emission_db.emissions_hotdeck_imputed`
WHERE imputation_priority = 'all_zero_missing_data'
GROUP BY imputation_priority, imputation_status
ORDER BY record_count DESC
"""

imputation_success = client.query(imputation_success_query).to_dataframe()
imputation_success


,imputation_priority,imputation_status,record_count,pct_within_priority
0,all_zero_missing_data,needs_further_imputation,1113477,99.999641
1,all_zero_missing_data,hotdeck_imputed,4,0.000359


In [ ]:
sample_imputed_query = """
SELECT
    global_id, year, global_fire_event_id,
    -- Original values
    ECO2, ECO, area_burned, consumed_fuel,
    -- Imputed values
    ECO2_hotdeck, ECO_hotdeck, area_burned_hotdeck, consumed_fuel_hotdeck,
    -- Donor values for comparison
    donor_ECO2, donor_ECO, donor_area_burned, donor_consumed_fuel,
    imputation_status, imputation_priority
FROM `code-for-planet.emission_db.emissions_hotdeck_imputed`
WHERE imputation_status = 'hotdeck_imputed'
LIMIT 10
"""

sample_imputed = client.query(sample_imputed_query).to_dataframe()
sample_imputed

,global_id,year,global_fire_event_id,ECO2,ECO,area_burned,consumed_fuel,ECO2_hotdeck,ECO_hotdeck,area_burned_hotdeck,consumed_fuel_hotdeck,donor_ECO2,donor_ECO,donor_area_burned,donor_consumed_fuel,imputation_status,imputation_priority
0,2012_656783,2012,2012_690703,0.0,0.0,0.0,0.0,876.307851,36.578145,62500.0,522.544932,876.307851,36.578145,62500.0,522.544932,hotdeck_imputed,all_zero_missing_data
1,2012_656796,2012,2012_690714,0.0,0.0,0.0,0.0,2381.256624,99.396520,62500.0,1419.950282,2381.256624,99.396520,62500.0,1419.950282,hotdeck_imputed,all_zero_missing_data
2,2007_70472,2007,2007_33516,0.0,0.0,0.0,0.0,633.716064,26.452072,62500.0,377.886740,633.716064,26.452072,62500.0,377.886740,hotdeck_imputed,all_zero_missing_data
3,2012_656787,2012,2012_690707,0.0,0.0,0.0,0.0,1822.145147,76.058533,62500.0,1086.550475,1822.145147,76.058533,62500.0,1086.550475,hotdeck_imputed,all_zero_missing_data


In [ ]:
knn_imputation_query = """
CREATE OR REPLACE TABLE `code-for-planet.emission_db.emissions_final_imputed` AS

WITH knn_donors AS (
    SELECT
        missing.global_id AS missing_record_id,
        AVG(donor.ECO2_hotdeck) AS knn_ECO2,
        AVG(donor.ECO_hotdeck) AS knn_ECO,
        AVG(donor.ECH4_hotdeck) AS knn_ECH4,
        AVG(donor.area_burned_hotdeck) AS knn_area_burned,
        AVG(donor.consumed_fuel_hotdeck) AS knn_consumed_fuel,
        COUNT(*) AS neighbor_count

    FROM `code-for-planet.emission_db.emissions_hotdeck_imputed` missing
    JOIN `code-for-planet.emission_db.emissions_hotdeck_imputed` donor
        ON missing.global_id != donor.global_id
        AND missing.year = donor.year
        AND donor.imputation_status IN ('hotdeck_imputed', 'original_data')
        AND missing.covertype = donor.covertype  -- Same vegetation type
        AND ABS(missing.fuel_moisture_class - donor.fuel_moisture_class) <= 2  -- Similar conditions
        -- Remove spatial distance for speed, use grid approximation instead
        AND ABS(missing.grid10k - donor.grid10k) <= 100  -- Roughly nearby grid cells

    WHERE missing.imputation_status = 'needs_further_imputation'

    GROUP BY missing.global_id
    HAVING COUNT(*) >= 3  -- Need at least 3 neighbors
)

SELECT
    e.*,
    kd.knn_ECO2,
    kd.knn_ECO,
    kd.knn_ECH4,
    kd.knn_area_burned,
    kd.knn_consumed_fuel,
    kd.neighbor_count,

    -- Final imputed values (hotdeck first, then KNN, then original)
    COALESCE(e.ECO2_hotdeck, kd.knn_ECO2, e.ECO2) AS ECO2_final,
    COALESCE(e.ECO_hotdeck, kd.knn_ECO, e.ECO) AS ECO_final,
    COALESCE(e.ECH4_hotdeck, kd.knn_ECH4, e.ECH4) AS ECH4_final,
    COALESCE(e.area_burned_hotdeck, kd.knn_area_burned, e.area_burned) AS area_burned_final,
    COALESCE(e.consumed_fuel_hotdeck, kd.knn_consumed_fuel, e.consumed_fuel) AS consumed_fuel_final,

    -- Final imputation method
    CASE
        WHEN e.imputation_status = 'original_data' THEN 'original'
        WHEN e.imputation_status = 'hotdeck_imputed' THEN 'hotdeck'
        WHEN kd.knn_ECO2 IS NOT NULL THEN 'knn'
        ELSE 'no_imputation_possible'
    END AS final_imputation_method

FROM `code-for-planet.emission_db.emissions_hotdeck_imputed` e
LEFT JOIN knn_donors kd ON e.global_id = kd.missing_record_id
"""

job = client.query(knn_imputation_query)
job.result()

In [ ]:
final_summary_query = """
SELECT
    final_imputation_method,
    COUNT(*) as record_count,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() as percentage,
    -- Show average values for positive emissions
    AVG(CASE WHEN ECO2_final > 0 THEN ECO2_final END) as avg_ECO2_final,
    AVG(CASE WHEN area_burned_final > 0 THEN area_burned_final END) as avg_area_burned_final,
    -- Count how many zeros remain
    COUNTIF(ECO2_final = 0) as remaining_ECO2_zeros,
    COUNTIF(area_burned_final = 0) as remaining_area_burned_zeros
FROM `code-for-planet.emission_db.emissions_final_imputed`
GROUP BY final_imputation_method
ORDER BY record_count DESC
"""

final_summary = client.query(final_summary_query).to_dataframe()
final_summary

,final_imputation_method,record_count,percentage,avg_ECO2_final,avg_area_burned_final,remaining_ECO2_zeros,remaining_area_burned_zeros
0,original,4761421,65.633726,2912.161777,62500.0,0,0
1,no_imputation_possible,1294924,17.849858,2776.685388,62500.0,1293939,1114462
2,knn,1198185,16.516361,3175.388381,62500.0,19,1198166
3,hotdeck,4,0.000055,1428.356422,62500.0,0,0


In [21]:
rescue_update_query = """
CREATE OR REPLACE TABLE `code-for-planet.emission_db.emissions_final_imputed` AS

WITH
-- Relaxed KNN for remaining zeros
relaxed_knn_rescue AS (
    SELECT
        missing.global_id,
        AVG(donor.ECO2_final) AS rescue_ECO2,
        AVG(donor.ECO_final) AS rescue_ECO,
        AVG(donor.ECH4_final) AS rescue_ECH4,
        AVG(donor.area_burned_final) AS rescue_area_burned,
        AVG(donor.consumed_fuel_final) AS rescue_consumed_fuel,
        COUNT(*) AS rescue_neighbors

    FROM `code-for-planet.emission_db.emissions_final_imputed` missing
    JOIN `code-for-planet.emission_db.emissions_final_imputed` donor
        ON missing.global_id != donor.global_id
        AND missing.year = donor.year
        AND donor.final_imputation_method IN ('original', 'knn', 'hotdeck')
        AND (missing.covertype = donor.covertype OR missing.fuelcode = donor.fuelcode)
        AND ABS(missing.fuel_moisture_class - donor.fuel_moisture_class) <= 5
        AND ABS(missing.grid10k - donor.grid10k) <= 1000

    WHERE missing.final_imputation_method = 'no_imputation_possible'

    GROUP BY missing.global_id
    HAVING COUNT(*) >= 2
),
-- Regional averages for records that can't find neighbors
regional_rescue AS (
    SELECT
        missing.global_id,
        ra.regional_ECO2,
        ra.regional_ECO,
        ra.regional_ECH4,
        ra.regional_area_burned,
        ra.regional_consumed_fuel
    FROM `code-for-planet.emission_db.emissions_final_imputed` missing
    LEFT JOIN relaxed_knn_rescue rkr ON missing.global_id = rkr.global_id
    JOIN (
        SELECT
            covertype, fuelcode,
            AVG(CASE WHEN ECO2_final > 0 THEN ECO2_final END) AS regional_ECO2,
            AVG(CASE WHEN ECO_final > 0 THEN ECO_final END) AS regional_ECO,
            AVG(CASE WHEN ECH4_final > 0 THEN ECH4_final END) AS regional_ECH4,
            AVG(CASE WHEN area_burned_final > 0 THEN area_burned_final END) AS regional_area_burned,
            AVG(CASE WHEN consumed_fuel_final > 0 THEN consumed_fuel_final END) AS regional_consumed_fuel
        FROM `code-for-planet.emission_db.emissions_final_imputed`
        WHERE final_imputation_method IN ('original', 'knn', 'hotdeck')
        GROUP BY covertype, fuelcode
        HAVING COUNT(CASE WHEN ECO2_final > 0 THEN 1 END) >= 10
    ) ra ON missing.covertype = ra.covertype AND missing.fuelcode = ra.fuelcode
    WHERE missing.final_imputation_method = 'no_imputation_possible'
    AND rkr.global_id IS NULL
)

SELECT
    -- All original columns EXCEPT the _final ones and final_imputation_method
    e.global_id,
    e.global_fire_event_id,
    e.original_id,
    e.year,
    e.doy,
    e.longitude,
    e.latitude,
    e.grid10k,
    e.covertype,
    e.fuelcode,
    e.area_burned,
    e.prefire_fuel,
    e.consumed_fuel,
    e.ECO2,
    e.ECO,
    e.ECH4,
    e.EPM2_5,
    e.cwd_frac,
    e.duff_frac,
    e.fuel_moisture_class,
    e.burn_source,
    e.burnday_source,
    e.BSEV,
    e.BSEV_flag,
    e.fire_date,
    e.bi_value,
    e.fm100_value,
    e.pet_value,
    e.fm1000_value,
    e.pr_value,
    e.rmax_value,
    e.rmin_value,
    e.sph_value,
    e.srad_value,
    e.tmmn_value,
    e.th_value,
    e.tmmx_value,
    e.vpd_value,
    e.vs_value,
    e.fire_region,
    e.original_fire_event_id,
    e.is_isolated_event,
    e.clustering_status,
    e.distance_to_event_centroid_km,
    e.days_from_event_start,
    e.event_size_points,
    e.event_duration_days,
    e.event_start_date,
    e.event_end_date,
    e.event_spatial_extent_km,
    e.event_centroid_lon,
    e.event_centroid_lat,
    e.event_total_ECO2,
    e.event_total_area_burned,
    e.event_spread_rate_km_per_day,
    e.fire_event_quality_flag,
    e.imputation_priority,
    e.zero_variable_count,

    -- Keep intermediate imputation columns if they exist
    e.donor_ECO2,
    e.donor_ECO,
    e.donor_ECH4,
    e.donor_area_burned,
    e.donor_consumed_fuel,
    e.ECO2_hotdeck,
    e.ECO_hotdeck,
    e.ECH4_hotdeck,
    e.area_burned_hotdeck,
    e.consumed_fuel_hotdeck,
    e.imputation_status,
    e.knn_ECO2,
    e.knn_ECO,
    e.knn_ECH4,
    e.knn_area_burned,
    e.knn_consumed_fuel,
    e.neighbor_count,

    -- Add rescue info
    rkr.rescue_neighbors,

    -- NEW FINAL VALUES (preserve existing, rescue zeros)
    CASE
        WHEN e.ECO2_final > 0 THEN e.ECO2_final
        WHEN rkr.rescue_ECO2 IS NOT NULL THEN rkr.rescue_ECO2
        WHEN rr.regional_ECO2 IS NOT NULL THEN rr.regional_ECO2
        ELSE e.ECO2_final
    END AS ECO2_final,

    CASE
        WHEN e.ECO_final > 0 THEN e.ECO_final
        WHEN rkr.rescue_ECO IS NOT NULL THEN rkr.rescue_ECO
        WHEN rr.regional_ECO IS NOT NULL THEN rr.regional_ECO
        ELSE e.ECO_final
    END AS ECO_final,

    CASE
        WHEN e.ECH4_final > 0 THEN e.ECH4_final
        WHEN rkr.rescue_ECH4 IS NOT NULL THEN rkr.rescue_ECH4
        WHEN rr.regional_ECH4 IS NOT NULL THEN rr.regional_ECH4
        ELSE e.ECH4_final
    END AS ECH4_final,

    CASE
        WHEN e.area_burned_final > 0 THEN e.area_burned_final
        WHEN rkr.rescue_area_burned IS NOT NULL THEN rkr.rescue_area_burned
        WHEN rr.regional_area_burned IS NOT NULL THEN rr.regional_area_burned
        ELSE e.area_burned_final
    END AS area_burned_final,

    CASE
        WHEN e.consumed_fuel_final > 0 THEN e.consumed_fuel_final
        WHEN rkr.rescue_consumed_fuel IS NOT NULL THEN rkr.rescue_consumed_fuel
        WHEN rr.regional_consumed_fuel IS NOT NULL THEN rr.regional_consumed_fuel
        ELSE e.consumed_fuel_final
    END AS consumed_fuel_final,

    -- NEW FINAL IMPUTATION METHOD
    CASE
        WHEN e.final_imputation_method != 'no_imputation_possible' THEN e.final_imputation_method
        WHEN rkr.rescue_ECO2 IS NOT NULL THEN 'rescued_relaxed_knn'
        WHEN rr.regional_ECO2 IS NOT NULL THEN 'rescued_regional_avg'
        ELSE 'no_imputation_possible'
    END AS final_imputation_method

FROM `code-for-planet.emission_db.emissions_final_imputed` e
LEFT JOIN relaxed_knn_rescue rkr ON e.global_id = rkr.global_id
LEFT JOIN regional_rescue rr ON e.global_id = rr.global_id
"""

job = client.query(rescue_update_query)
result = job.result()





In [22]:
rescue_results_query = """
SELECT
    final_imputation_method,
    COUNT(*) as record_count,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() as percentage
FROM `code-for-planet.emission_db.emissions_final_imputed`
GROUP BY final_imputation_method
ORDER BY record_count DESC
"""

rescue_results = client.query(rescue_results_query).to_dataframe()
rescue_results

,final_imputation_method,record_count,percentage
0,original,4761421,65.633726
1,no_imputation_possible,1293936,17.836239
2,knn,1198185,16.516361
3,rescued_relaxed_knn,984,0.013564
4,rescued_regional_avg,4,0.000055
5,hotdeck,4,0.000055


In [23]:
final_zero_counts_query = """
SELECT
    COUNT(*) as total_records,

    -- Final values zero counts (variables we imputed)
    COUNTIF(area_burned_final = 0) as area_burned_final_zeros,
    COUNTIF(consumed_fuel_final = 0) as consumed_fuel_final_zeros,
    COUNTIF(ECO2_final = 0) as ECO2_final_zeros,
    COUNTIF(ECO_final = 0) as ECO_final_zeros,
    COUNTIF(ECH4_final = 0) as ECH4_final_zeros,

    -- Original columns that don't have _final versions
    COUNTIF(prefire_fuel = 0) as prefire_fuel_zeros,
    COUNTIF(EPM2_5 = 0) as EPM2_5_zeros,
    COUNTIF(cwd_frac = 0) as cwd_frac_zeros,
    COUNTIF(duff_frac = 0) as duff_frac_zeros,
    COUNTIF(BSEV = 0) as BSEV_zeros,

    -- Additional fire indicators
    COUNTIF(burn_source = 0) as burn_source_zeros,
    COUNTIF(burnday_source = 0) as burnday_source_zeros,
    COUNTIF(BSEV_flag = 0) as BSEV_flag_zeros,
    COUNTIF(fuel_moisture_class = 0) as fuel_moisture_class_zeros

FROM `code-for-planet.emission_db.emissions_final_imputed`
"""

final_zero_counts = client.query(final_zero_counts_query).to_dataframe()
final_zero_counts

,total_records,area_burned_final_zeros,consumed_fuel_final_zeros,ECO2_final_zeros,ECO_final_zeros,ECH4_final_zeros,prefire_fuel_zeros,EPM2_5_zeros,cwd_frac_zeros,duff_frac_zeros,BSEV_zeros,burn_source_zeros,burnday_source_zeros,BSEV_flag_zeros,fuel_moisture_class_zeros
0,7254534,2311643,1293955,1293955,1293955,1293955,1293962,1293962,4472065,4472065,1062445,0,0,5394145,0


In [26]:
rescue_query = """
CREATE OR REPLACE TABLE `code-for-planet.emission_db.emissions_final_imputed` AS

WITH yearly_averages AS (
    -- Calculate simple yearly averages from good data
    SELECT
        year,
        AVG(CASE WHEN ECO2_final > 0 THEN ECO2_final END) as year_avg_ECO2,
        AVG(CASE WHEN ECO_final > 0 THEN ECO_final END) as year_avg_ECO,
        AVG(CASE WHEN ECH4_final > 0 THEN ECH4_final END) as year_avg_ECH4,
        AVG(CASE WHEN area_burned_final > 0 THEN area_burned_final END) as year_avg_area_burned,
        AVG(CASE WHEN consumed_fuel_final > 0 THEN consumed_fuel_final END) as year_avg_consumed_fuel
    FROM `code-for-planet.emission_db.emissions_final_imputed`
    WHERE final_imputation_method IN ('original', 'knn', 'rescued_relaxed_knn', 'rescued_regional_avg')
    GROUP BY year
)

SELECT
    -- Keep all original columns
    e.global_id, e.global_fire_event_id, e.original_id, e.year, e.doy, e.longitude, e.latitude,
    e.grid10k, e.covertype, e.fuelcode, e.area_burned, e.prefire_fuel, e.consumed_fuel,
    e.ECO2, e.ECO, e.ECH4, e.EPM2_5, e.cwd_frac, e.duff_frac, e.fuel_moisture_class,
    e.burn_source, e.burnday_source, e.BSEV, e.BSEV_flag, e.fire_date, e.bi_value,
    e.fm100_value, e.pet_value, e.fm1000_value, e.pr_value, e.rmax_value, e.rmin_value,
    e.sph_value, e.srad_value, e.tmmn_value, e.th_value, e.tmmx_value, e.vpd_value,
    e.vs_value, e.fire_region, e.original_fire_event_id, e.is_isolated_event,
    e.clustering_status, e.distance_to_event_centroid_km, e.days_from_event_start,
    e.event_size_points, e.event_duration_days, e.event_start_date, e.event_end_date,
    e.event_spatial_extent_km, e.event_centroid_lon, e.event_centroid_lat,
    e.event_total_ECO2, e.event_total_area_burned, e.event_spread_rate_km_per_day,
    e.fire_event_quality_flag, e.imputation_priority, e.zero_variable_count,

    -- SIMPLE RESCUE: Use yearly averages for all zeros
    CASE
        WHEN e.ECO2_final > 0 THEN e.ECO2_final
        WHEN ya.year_avg_ECO2 IS NOT NULL THEN ya.year_avg_ECO2
        ELSE 100.0  -- Last resort
    END AS ECO2_final,

    CASE
        WHEN e.ECO_final > 0 THEN e.ECO_final
        WHEN ya.year_avg_ECO IS NOT NULL THEN ya.year_avg_ECO
        ELSE 50.0
    END AS ECO_final,

    CASE
        WHEN e.ECH4_final > 0 THEN e.ECH4_final
        WHEN ya.year_avg_ECH4 IS NOT NULL THEN ya.year_avg_ECH4
        ELSE 5.0
    END AS ECH4_final,

    CASE
        WHEN e.area_burned_final > 0 THEN e.area_burned_final
        WHEN ya.year_avg_area_burned IS NOT NULL THEN ya.year_avg_area_burned
        ELSE 0.1
    END AS area_burned_final,

    CASE
        WHEN e.consumed_fuel_final > 0 THEN e.consumed_fuel_final
        WHEN ya.year_avg_consumed_fuel IS NOT NULL THEN ya.year_avg_consumed_fuel
        ELSE 10.0
    END AS consumed_fuel_final,

    -- Update method tracking
    CASE
        WHEN e.final_imputation_method != 'no_imputation_possible' THEN e.final_imputation_method
        WHEN ya.year_avg_ECO2 IS NOT NULL THEN 'rescued_simple_year_avg'
        ELSE 'rescued_minimum_estimate'
    END AS final_imputation_method

FROM `code-for-planet.emission_db.emissions_final_imputed` e
LEFT JOIN yearly_averages ya ON e.year = ya.year
"""

job = client.query(rescue_query)
result = job.result()

In [27]:
final_results_query = """
SELECT
    final_imputation_method,
    COUNT(*) as record_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage,
    ROUND(AVG(ECO2_final), 2) as avg_ECO2_final,
    ROUND(AVG(area_burned_final), 4) as avg_area_burned_final,
    -- Check for remaining zeros (should be very few!)
    COUNTIF(ECO2_final = 0) as remaining_ECO2_zeros,
    COUNTIF(area_burned_final = 0) as remaining_area_burned_zeros,
    COUNTIF(ECO_final = 0) as remaining_ECO_zeros,
    COUNTIF(ECH4_final = 0) as remaining_ECH4_zeros,
    COUNTIF(consumed_fuel_final = 0) as remaining_consumed_fuel_zeros
FROM `code-for-planet.emission_db.emissions_final_imputed`
GROUP BY final_imputation_method
ORDER BY record_count DESC
"""

final_results = client.query(final_results_query).to_dataframe()
final_results

,final_imputation_method,record_count,percentage,avg_ECO2_final,avg_area_burned_final,remaining_ECO2_zeros,remaining_area_burned_zeros,remaining_ECO_zeros,remaining_ECH4_zeros,remaining_consumed_fuel_zeros
0,original,4761421,65.63,2912.16,62500.0000,0,0,0,0,0
1,rescued_simple_year_avg,1293936,17.84,3107.05,62497.2296,0,0,0,0,0
2,knn,1198185,16.52,3175.39,62497.3211,0,0,0,0,0
3,rescued_relaxed_knn,984,0.01,2775.18,50212.3457,0,0,0,0,0
4,hotdeck,4,0.00,1428.36,62500.0000,0,0,0,0,0
5,rescued_regional_avg,4,0.00,1774.52,62500.0000,0,0,0,0,0


In [28]:
zero_check_query = """
SELECT
    COUNT(*) as total_records,
    COUNTIF(ECO2_final = 0) as ECO2_final_zeros,
    COUNTIF(ECO_final = 0) as ECO_final_zeros,
    COUNTIF(ECH4_final = 0) as ECH4_final_zeros,
    COUNTIF(area_burned_final = 0) as area_burned_final_zeros,
    COUNTIF(consumed_fuel_final = 0) as consumed_fuel_final_zeros,

    -- Percentages
    ROUND(COUNTIF(ECO2_final = 0) * 100.0 / COUNT(*), 3) as ECO2_zero_pct,
    ROUND(COUNTIF(area_burned_final = 0) * 100.0 / COUNT(*), 3) as area_burned_zero_pct
FROM `code-for-planet.emission_db.emissions_final_imputed`
"""

zero_check = client.query(zero_check_query).to_dataframe()
zero_check

,total_records,ECO2_final_zeros,ECO_final_zeros,ECH4_final_zeros,area_burned_final_zeros,consumed_fuel_final_zeros,ECO2_zero_pct,area_burned_zero_pct
0,7254534,0,0,0,0,0,0.0,0.0


In [29]:
all_zeros_count_query = """
SELECT
    COUNT(*) as total_records,

    -- ID and basic info columns
    COUNTIF(original_id = 0) as original_id_zeros,
    COUNTIF(year = 0) as year_zeros,
    COUNTIF(doy = 0) as doy_zeros,
    COUNTIF(longitude = 0) as longitude_zeros,
    COUNTIF(latitude = 0) as latitude_zeros,
    COUNTIF(grid10k = 0) as grid10k_zeros,

    -- Fire characteristic columns
    COUNTIF(covertype = 0) as covertype_zeros,
    COUNTIF(fuelcode = 0) as fuelcode_zeros,
    COUNTIF(fuel_moisture_class = 0) as fuel_moisture_class_zeros,
    COUNTIF(burn_source = 0) as burn_source_zeros,
    COUNTIF(burnday_source = 0) as burnday_source_zeros,
    COUNTIF(BSEV = 0) as BSEV_zeros,
    COUNTIF(BSEV_flag = 0) as BSEV_flag_zeros,

    -- Original emission/fuel columns (before imputation)
    COUNTIF(area_burned = 0) as area_burned_original_zeros,
    COUNTIF(prefire_fuel = 0) as prefire_fuel_zeros,
    COUNTIF(consumed_fuel = 0) as consumed_fuel_original_zeros,
    COUNTIF(ECO2 = 0) as ECO2_original_zeros,
    COUNTIF(ECO = 0) as ECO_original_zeros,
    COUNTIF(ECH4 = 0) as ECH4_original_zeros,
    COUNTIF(EPM2_5 = 0) as EPM2_5_zeros,
    COUNTIF(cwd_frac = 0) as cwd_frac_zeros,
    COUNTIF(duff_frac = 0) as duff_frac_zeros,

    -- Final imputed columns (should be zero for emission variables)
    COUNTIF(area_burned_final = 0) as area_burned_final_zeros,
    COUNTIF(consumed_fuel_final = 0) as consumed_fuel_final_zeros,
    COUNTIF(ECO2_final = 0) as ECO2_final_zeros,
    COUNTIF(ECO_final = 0) as ECO_final_zeros,
    COUNTIF(ECH4_final = 0) as ECH4_final_zeros

FROM `code-for-planet.emission_db.emissions_final_imputed`
"""

all_zeros_count = client.query(all_zeros_count_query).to_dataframe()
all_zeros_count

,total_records,original_id_zeros,year_zeros,doy_zeros,longitude_zeros,latitude_zeros,grid10k_zeros,covertype_zeros,fuelcode_zeros,fuel_moisture_class_zeros,...,ECO_original_zeros,ECH4_original_zeros,EPM2_5_zeros,cwd_frac_zeros,duff_frac_zeros,area_burned_final_zeros,consumed_fuel_final_zeros,ECO2_final_zeros,ECO_final_zeros,ECH4_final_zeros
0,7254534,11,0,25,0,0,14,134288,1293936,0,...,1293962,1293962,1293962,4472065,4472065,0,0,0,0,0


In [30]:
all_zeros_percentage_query = """
SELECT
    -- ID and basic info columns
    ROUND(COUNTIF(original_id = 0) * 100.0 / COUNT(*), 3) as original_id_zero_pct,
    ROUND(COUNTIF(year = 0) * 100.0 / COUNT(*), 3) as year_zero_pct,
    ROUND(COUNTIF(doy = 0) * 100.0 / COUNT(*), 3) as doy_zero_pct,
    ROUND(COUNTIF(longitude = 0) * 100.0 / COUNT(*), 3) as longitude_zero_pct,
    ROUND(COUNTIF(latitude = 0) * 100.0 / COUNT(*), 3) as latitude_zero_pct,
    ROUND(COUNTIF(grid10k = 0) * 100.0 / COUNT(*), 3) as grid10k_zero_pct,

    -- Fire characteristic columns
    ROUND(COUNTIF(covertype = 0) * 100.0 / COUNT(*), 3) as covertype_zero_pct,
    ROUND(COUNTIF(fuelcode = 0) * 100.0 / COUNT(*), 3) as fuelcode_zero_pct,
    ROUND(COUNTIF(fuel_moisture_class = 0) * 100.0 / COUNT(*), 3) as fuel_moisture_class_zero_pct,
    ROUND(COUNTIF(burn_source = 0) * 100.0 / COUNT(*), 3) as burn_source_zero_pct,
    ROUND(COUNTIF(burnday_source = 0) * 100.0 / COUNT(*), 3) as burnday_source_zero_pct,
    ROUND(COUNTIF(BSEV = 0) * 100.0 / COUNT(*), 3) as BSEV_zero_pct,
    ROUND(COUNTIF(BSEV_flag = 0) * 100.0 / COUNT(*), 3) as BSEV_flag_zero_pct,

    -- Original emission/fuel columns
    ROUND(COUNTIF(area_burned = 0) * 100.0 / COUNT(*), 3) as area_burned_original_zero_pct,
    ROUND(COUNTIF(prefire_fuel = 0) * 100.0 / COUNT(*), 3) as prefire_fuel_zero_pct,
    ROUND(COUNTIF(consumed_fuel = 0) * 100.0 / COUNT(*), 3) as consumed_fuel_original_zero_pct,
    ROUND(COUNTIF(ECO2 = 0) * 100.0 / COUNT(*), 3) as ECO2_original_zero_pct,
    ROUND(COUNTIF(ECO = 0) * 100.0 / COUNT(*), 3) as ECO_original_zero_pct,
    ROUND(COUNTIF(ECH4 = 0) * 100.0 / COUNT(*), 3) as ECH4_original_zero_pct,
    ROUND(COUNTIF(EPM2_5 = 0) * 100.0 / COUNT(*), 3) as EPM2_5_zero_pct,
    ROUND(COUNTIF(cwd_frac = 0) * 100.0 / COUNT(*), 3) as cwd_frac_zero_pct,
    ROUND(COUNTIF(duff_frac = 0) * 100.0 / COUNT(*), 3) as duff_frac_zero_pct,

    -- Final imputed columns
    ROUND(COUNTIF(area_burned_final = 0) * 100.0 / COUNT(*), 3) as area_burned_final_zero_pct,
    ROUND(COUNTIF(consumed_fuel_final = 0) * 100.0 / COUNT(*), 3) as consumed_fuel_final_zero_pct,
    ROUND(COUNTIF(ECO2_final = 0) * 100.0 / COUNT(*), 3) as ECO2_final_zero_pct,
    ROUND(COUNTIF(ECO_final = 0) * 100.0 / COUNT(*), 3) as ECO_final_zero_pct,
    ROUND(COUNTIF(ECH4_final = 0) * 100.0 / COUNT(*), 3) as ECH4_final_zero_pct,

    COUNT(*) as total_records
FROM `code-for-planet.emission_db.emissions_final_imputed`
"""

all_zeros_percentage = client.query(all_zeros_percentage_query).to_dataframe()
all_zeros_percentage

,original_id_zero_pct,year_zero_pct,doy_zero_pct,longitude_zero_pct,latitude_zero_pct,grid10k_zero_pct,covertype_zero_pct,fuelcode_zero_pct,fuel_moisture_class_zero_pct,burn_source_zero_pct,...,ECH4_original_zero_pct,EPM2_5_zero_pct,cwd_frac_zero_pct,duff_frac_zero_pct,area_burned_final_zero_pct,consumed_fuel_final_zero_pct,ECO2_final_zero_pct,ECO_final_zero_pct,ECH4_final_zero_pct,total_records
0,0.0,0.0,0.0,0.0,0.0,0.0,1.851,17.836,0.0,0.0,...,17.837,17.837,61.645,61.645,0.0,0.0,0.0,0.0,0.0,7254534


In [31]:
null_check_query = """
SELECT
    COUNT(*) as total_records,

    -- Check for NULLs in key columns
    COUNTIF(global_id IS NULL) as global_id_nulls,
    COUNTIF(global_fire_event_id IS NULL) as global_fire_event_id_nulls,
    COUNTIF(original_id IS NULL) as original_id_nulls,
    COUNTIF(year IS NULL) as year_nulls,
    COUNTIF(doy IS NULL) as doy_nulls,
    COUNTIF(longitude IS NULL) as longitude_nulls,
    COUNTIF(latitude IS NULL) as latitude_nulls,
    COUNTIF(covertype IS NULL) as covertype_nulls,
    COUNTIF(fuelcode IS NULL) as fuelcode_nulls,
    COUNTIF(fire_date IS NULL) as fire_date_nulls,

    -- Check for NULLs in emission variables
    COUNTIF(ECO2_final IS NULL) as ECO2_final_nulls,
    COUNTIF(area_burned_final IS NULL) as area_burned_final_nulls,
    COUNTIF(prefire_fuel IS NULL) as prefire_fuel_nulls,
    COUNTIF(EPM2_5 IS NULL) as EPM2_5_nulls,
    COUNTIF(cwd_frac IS NULL) as cwd_frac_nulls,
    COUNTIF(duff_frac IS NULL) as duff_frac_nulls,
    COUNTIF(BSEV IS NULL) as BSEV_nulls

FROM `code-for-planet.emission_db.emissions_final_imputed`
"""

null_check = client.query(null_check_query).to_dataframe()
null_check

,total_records,global_id_nulls,global_fire_event_id_nulls,original_id_nulls,year_nulls,doy_nulls,longitude_nulls,latitude_nulls,covertype_nulls,fuelcode_nulls,fire_date_nulls,ECO2_final_nulls,area_burned_final_nulls,prefire_fuel_nulls,EPM2_5_nulls,cwd_frac_nulls,duff_frac_nulls,BSEV_nulls
0,7254534,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
